In [991]:
import os
import numpy as np
import pandas as pd

In [992]:

file_path_1 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/fundamental_all.csv' # Fundamental data
df1 = pd.read_csv(file_path_1)
df1['datadate'] = pd.to_datetime(df1['datadate'])
df1 = df1.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr','costat'])

df1 = df1.sort_values(by=['tic', 'datadate'])

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17745 entries, 1236 to 15067
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     17745 non-null  int64         
 1   datadate  17745 non-null  datetime64[ns]
 2   tic       17745 non-null  object        
 3   actq      13578 non-null  float64       
 4   ancq      13192 non-null  float64       
 5   epsfxq    16905 non-null  float64       
 6   lctq      13579 non-null  float64       
 7   ltq       15674 non-null  float64       
 8   revtq     17110 non-null  float64       
 9   ugiq      496 non-null    float64       
 10  capxy     13254 non-null  float64       
 11  prccq     17377 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 1.8+ MB


In [993]:
df1.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,ugiq,capxy,prccq
1236,1690,1979-12-31,AAPL,23.052,1.584,0.06,11.689,12.262,19.540,NaN,NaN,NaN
1237,1690,1980-03-31,AAPL,28.367,1.986,0.06,14.244,14.632,23.550,NaN,NaN,NaN
1238,1690,1980-06-30,AAPL,37.930,3.777,0.06,20.976,21.926,32.569,NaN,NaN,NaN
1239,1690,1980-09-30,AAPL,54.106,11.244,0.07,37.780,39.401,41.467,NaN,NaN,NaN
1240,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,NaN,34.124969


In [994]:
file_path_2 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/ratios_all.csv' # Financial ratios data
df2 = pd.read_csv(file_path_2)
df2['qdate'] = pd.to_datetime(df2['qdate'])
df2['public_date'] = pd.to_datetime(df2['public_date'])

# Sort by qdate and public_date to ensure the most recent public_date comes last
df2 = df2.sort_values(by=['qdate', 'public_date'])
# Drop duplicates to keep only the first public_date for each qdate
df2 = df2.drop_duplicates(subset=['qdate','TICKER'], keep='first')

columns_to_keep = [
    'gvkey', 'qdate', 'roa',
    'roe','pe_inc', 'divyield',
    'pcf', 'roce', 'fcf_ocf',
    'debt_ebitda', 'gpm', 'opmad',
    'evm', 'intcov_ratio', 'at_turn',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb',
]

df2 = df2[columns_to_keep]

# convert divyield to float
df2['divyield'] = df2['divyield'].replace('%', '', regex=True).astype(float)

df2 = df2.sort_values(by=['qdate'])

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15533 entries, 0 to 26692
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gvkey         15533 non-null  int64         
 1   qdate         15520 non-null  datetime64[ns]
 2   roa           15472 non-null  float64       
 3   roe           15267 non-null  float64       
 4   pe_inc        14787 non-null  float64       
 5   divyield      11974 non-null  float64       
 6   pcf           15467 non-null  float64       
 7   roce          15353 non-null  float64       
 8   fcf_ocf       14206 non-null  float64       
 9   debt_ebitda   15355 non-null  float64       
 10  gpm           15512 non-null  float64       
 11  opmad         15512 non-null  float64       
 12  evm           15437 non-null  float64       
 13  intcov_ratio  14432 non-null  float64       
 14  at_turn       15475 non-null  float64       
 15  cfm           15197 non-null  float6

In [995]:
df = pd.merge(df1, df2, left_on=['datadate', 'gvkey'], right_on=['qdate', 'gvkey'], how='inner')

columns_to_keep = [
    'gvkey', 'datadate', 'tic',
    'actq', 'ancq', 'epsfxq',
    'lctq', 'ltq', 'revtq',
    'capxy', 'prccq', 'roa',
    'roe','pe_inc', 'divyield',
    'pcf', 'roce', 'debt_ebitda',
    'gpm', 'evm', 'cfm', 'npm',
    'PEG_trailing', 'de_ratio',
    'curr_ratio', 'ptb',
]

column_renames = {
    'datadate': 'date', 
    'PRC': 'price', 
    'epsfxq': 'earnings_per_share', 
    'ltq': 'long_term_debt', 
    'niq': 'net_income', 
    'revtq': 'revenue', 
    'capxy': 'capital_expenditure', 
    'npm': 'net_profit_margin', 
    'de_ratio': 'debt/equity_Ratio', 
    'curr_ratio': 'current_ratio', 
    'ptb': 'price/book_ratio',
    'tic': 'ticker',
    'actq': 'current_assets',
    'ancq': 'non_current_assets',
    'lctq': 'total_liability',
    'prccq': 'price',
    'roa': 'return_on_assets',
    'roe': 'return_on_equity',
    'pe_inc': 'price/equity_ratio',
    'divyield': 'dividend_yield',
    'pcf': 'price/cash_flow',
    'roce': 'return_on_capital_employed',
    'debt_ebitda': 'debt/ebitda_ratio',
    'gpm': 'gross_profit_margin',
    'evm': 'enterprise_value_multiple',
    'cfm': 'cash_flow_margin',
    'PEG_trailing': 'trailing_p/e_to_growth_ratio'
}

df = df[columns_to_keep]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15523 entries, 0 to 15522
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gvkey         15523 non-null  int64         
 1   datadate      15523 non-null  datetime64[ns]
 2   tic           15523 non-null  object        
 3   actq          12739 non-null  float64       
 4   ancq          12361 non-null  float64       
 5   epsfxq        15404 non-null  float64       
 6   lctq          12742 non-null  float64       
 7   ltq           14742 non-null  float64       
 8   revtq         15076 non-null  float64       
 9   capxy         12419 non-null  float64       
 10  prccq         15502 non-null  float64       
 11  roa           15469 non-null  float64       
 12  roe           15264 non-null  float64       
 13  pe_inc        14789 non-null  float64       
 14  divyield      11974 non-null  float64       
 15  pcf           15464 non-null  float6

In [996]:
df = df.sort_values(by=['tic','datadate'])

df.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,prccq,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb
0,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,34.124969,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083
1,1690,1981-03-31,AAPL,173.152,17.587,0.16,39.428,43.693,78.765,NaN,24.499985,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188
2,1690,1981-06-30,AAPL,201.262,21.085,0.21,49.914,58.268,90.719,NaN,25.999985,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507
3,1690,1981-09-30,AAPL,227.104,27.734,0.19,70.280,77.451,97.678,NaN,15.249996,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659
4,1690,1981-12-31,AAPL,249.160,36.026,0.24,83.952,92.894,133.553,NaN,22.124985,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086


In [997]:
file_path_3 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/s&p500index.csv' # s&p500
df3 = pd.read_csv(file_path_3)
df3['caldt'] = pd.to_datetime(df3['caldt'])  # Adjust column name if needed

df = df.sort_values(by=['datadate'])


#merge (fill empty cells with nearest s&p500 value)
df = pd.merge_asof(df, df3, left_on='datadate', right_on='caldt', direction='nearest')
df = df.drop(columns=['caldt'])

# calculate relative returns
#df['relative_quarterly_return'] = (df['prccq'] - df['spindx'])
#df = df.drop(columns=['prccq', 'spindx'])

#df = df.dropna(subset=['prccq'])

df = df.sort_values(by=['tic','datadate']).reset_index()
df.head()

,index,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,prccq,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,spindx
0,2010,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,34.124969,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,135.76
1,2067,1690,1981-03-31,AAPL,173.152,17.587,0.16,39.428,43.693,78.765,NaN,24.499985,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,136.00
2,2115,1690,1981-06-30,AAPL,201.262,21.085,0.21,49.914,58.268,90.719,NaN,25.999985,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,131.21
3,2132,1690,1981-09-30,AAPL,227.104,27.734,0.19,70.280,77.451,97.678,NaN,15.249996,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,116.18
4,2208,1690,1981-12-31,AAPL,249.160,36.026,0.24,83.952,92.894,133.553,NaN,22.124985,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,122.55


In [998]:


# convert features to pct

features = [
    'actq', 'ancq', 'epsfxq',
    'lctq', 'ltq', 'revtq',
    'capxy', 'prccq',]

ratios = [
    'roa', 'roe','pe_inc', 'divyield',
    'pcf', 'roce',
    'debt_ebitda', 'gpm','evm',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb']

df[features] = df[features].pct_change()

for col in ratios:
    df[col + '_change'] = df[col].pct_change()


df['spindx'] = df['spindx'].pct_change()

df['relative_quarterly_return'] = (df['prccq'] - df['spindx'])
df['target'] = df['relative_quarterly_return'].shift(-1)

df = df.drop(columns=['prccq', 'spindx'])

df.head(200)

,index,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,roa_change,roe_change,pe_inc_change,divyield_change,pcf_change,roce_change,debt_ebitda_change,gpm_change,evm_change,cfm_change,npm_change,PEG_trailing_change,de_ratio_change,curr_ratio_change,ptb_change,relative_quarterly_return,target
0,2010,1690,1980-12-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.283819
1,2067,1690,1981-03-31,AAPL,0.188251,0.072247,0.142857,0.365898,0.404152,0.164801,NaN,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,-0.287897,-0.460829,-0.040696,NaN,0.147918,0.279839,-0.259259,0.016166,0.311636,0.000000,0.040000,NaN,-0.244604,0.238132,0.099702,-0.283819,0.096445
2,2115,1690,1981-06-30,AAPL,0.162343,0.198897,0.312500,0.265953,0.333577,0.151768,NaN,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,-0.110561,-0.170940,-0.549581,NaN,-0.333208,0.345936,0.400000,0.052273,0.058469,0.000000,0.105769,NaN,-0.123810,0.138219,-0.466114,0.096445,-0.298912
3,2132,1690,1981-09-30,AAPL,0.128400,0.315343,-0.095238,0.408022,0.329220,0.076709,NaN,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,-0.133581,-0.030928,-0.233183,NaN,-0.213799,-0.727060,5.500000,0.051836,-0.389708,0.276786,0.026087,NaN,0.187500,-0.122249,-0.130321,-0.298912,0.395990
4,2208,1690,1981-12-31,AAPL,0.097119,0.298983,0.263158,0.194536,0.199391,0.367278,NaN,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,-0.124197,-0.239362,-0.142632,NaN,3.038277,-0.264151,-0.829670,0.032854,0.248343,-0.020979,-0.033898,NaN,-0.084668,0.080780,-0.101255,0.395990,-0.150875
5,2278,1690,1982-03-31,AAPL,0.056109,0.089463,0.000000,-0.138794,-0.083902,-0.018974,NaN,0.402,0.285,15.909,NaN,31.409,0.479,0.028,0.519,11.643,0.139,0.111,NaN,0.418,3.403,3.478,-0.017115,-0.003497,-0.302604,NaN,-1.173895,0.116550,-0.096774,0.031809,-0.186714,-0.007143,-0.026316,NaN,0.045000,-0.025487,-0.316162,-0.150875,-0.223454
6,2318,1690,1982-06-30,AAPL,0.077134,0.032485,0.083333,0.056390,0.050846,0.089010,NaN,0.390,0.274,19.355,NaN,21.844,0.507,0.027,0.530,9.045,0.138,0.106,NaN,0.420,3.377,4.176,-0.029851,-0.038596,0.216607,NaN,-0.304531,0.058455,-0.035714,0.021195,-0.223138,-0.007194,-0.045045,NaN,0.004785,-0.007640,0.200690,-0.223454,0.332749
7,2357,1690,1982-09-30,AAPL,0.118679,0.004639,0.230769,0.122799,0.126002,0.232175,NaN,0.388,0.271,30.071,NaN,22.712,0.449,0.062,0.534,8.840,0.134,0.105,NaN,0.392,3.697,6.792,-0.005128,-0.010949,0.553655,NaN,0.039736,-0.114398,1.296296,0.007547,-0.022664,-0.028986,-0.009434,NaN,-0.066667,0.094759,0.626437,0.332749,0.469075
8,2419,1690,1982-12-31,AAPL,0.138737,0.156711,0.250000,0.106348,0.175967,0.218904,NaN,0.437,0.302,37.398,NaN,25.365,0.461,0.023,0.547,8.025,0.135,0.107,NaN,0.394,3.719,8.444,0.126289,0.114391,0.243657,NaN,0.116810,0.026726,-0.629032,0.024345,-0.092195,0.007463,0.019048,NaN,0.005102,0.005951,0.243227,0.469075,0.326626
9,2501,1690,1983-03-31,AAPL,0.127908,0.108660,0.000000,0.086197,0.129199,0.063880,NaN,0.457,0.308,41.848,NaN,34.438,0.534,0.020,0.547,9.364,0.133,0.107,NaN,0.399,3.802,9.489,0.045767,0.019868,0.118990,NaN,0.357698,0.158351,-0.130435,0.000000,0.166854,-0.014815,0.000000,NaN,0.012690,0.022318,0.123757,0.326626,0.057759


In [999]:
# REMOVE FALSE VALUES CAUSED BY PCT CHANGE

# Create a list to track the indices to drop
indices_to_drop = []

# Initialize the previous ticker as None
previous_ticker = None

# Iterate through the DataFrame, skipping the first row
for idx, row in df.iterrows():
    current_ticker = row['tic']
    
    # If the current ticker is different from the previous one, mark the current row for deletion
    if previous_ticker is not None and current_ticker != previous_ticker:
        indices_to_drop.append(idx)  # Append the index of the first occurrence of the new ticker
        indices_to_drop.append(idx-1)  # Append the index of the first occurrence of the new ticker
    
    # Update the previous ticker
    previous_ticker = current_ticker

# Drop the rows with the indices in the list
df = df.drop(indices_to_drop)

# Reset the index of the DataFrame after dropping rows
df = df.reset_index(drop=True)

df = df.drop(columns = ['index'])

df.head(200)

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,roa_change,roe_change,pe_inc_change,divyield_change,pcf_change,roce_change,debt_ebitda_change,gpm_change,evm_change,cfm_change,npm_change,PEG_trailing_change,de_ratio_change,curr_ratio_change,ptb_change,relative_quarterly_return,target
0,1690,1980-12-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.283819
1,1690,1981-03-31,AAPL,0.188251,0.072247,0.142857,0.365898,0.404152,0.164801,NaN,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,-0.287897,-0.460829,-0.040696,NaN,0.147918,0.279839,-0.259259,0.016166,0.311636,0.000000,0.040000,NaN,-0.244604,0.238132,0.099702,-0.283819,0.096445
2,1690,1981-06-30,AAPL,0.162343,0.198897,0.312500,0.265953,0.333577,0.151768,NaN,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,-0.110561,-0.170940,-0.549581,NaN,-0.333208,0.345936,0.400000,0.052273,0.058469,0.000000,0.105769,NaN,-0.123810,0.138219,-0.466114,0.096445,-0.298912
3,1690,1981-09-30,AAPL,0.128400,0.315343,-0.095238,0.408022,0.329220,0.076709,NaN,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,-0.133581,-0.030928,-0.233183,NaN,-0.213799,-0.727060,5.500000,0.051836,-0.389708,0.276786,0.026087,NaN,0.187500,-0.122249,-0.130321,-0.298912,0.395990
4,1690,1981-12-31,AAPL,0.097119,0.298983,0.263158,0.194536,0.199391,0.367278,NaN,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,-0.124197,-0.239362,-0.142632,NaN,3.038277,-0.264151,-0.829670,0.032854,0.248343,-0.020979,-0.033898,NaN,-0.084668,0.080780,-0.101255,0.395990,-0.150875
5,1690,1982-03-31,AAPL,0.056109,0.089463,0.000000,-0.138794,-0.083902,-0.018974,NaN,0.402,0.285,15.909,NaN,31.409,0.479,0.028,0.519,11.643,0.139,0.111,NaN,0.418,3.403,3.478,-0.017115,-0.003497,-0.302604,NaN,-1.173895,0.116550,-0.096774,0.031809,-0.186714,-0.007143,-0.026316,NaN,0.045000,-0.025487,-0.316162,-0.150875,-0.223454
6,1690,1982-06-30,AAPL,0.077134,0.032485,0.083333,0.056390,0.050846,0.089010,NaN,0.390,0.274,19.355,NaN,21.844,0.507,0.027,0.530,9.045,0.138,0.106,NaN,0.420,3.377,4.176,-0.029851,-0.038596,0.216607,NaN,-0.304531,0.058455,-0.035714,0.021195,-0.223138,-0.007194,-0.045045,NaN,0.004785,-0.007640,0.200690,-0.223454,0.332749
7,1690,1982-09-30,AAPL,0.118679,0.004639,0.230769,0.122799,0.126002,0.232175,NaN,0.388,0.271,30.071,NaN,22.712,0.449,0.062,0.534,8.840,0.134,0.105,NaN,0.392,3.697,6.792,-0.005128,-0.010949,0.553655,NaN,0.039736,-0.114398,1.296296,0.007547,-0.022664,-0.028986,-0.009434,NaN,-0.066667,0.094759,0.626437,0.332749,0.469075
8,1690,1982-12-31,AAPL,0.138737,0.156711,0.250000,0.106348,0.175967,0.218904,NaN,0.437,0.302,37.398,NaN,25.365,0.461,0.023,0.547,8.025,0.135,0.107,NaN,0.394,3.719,8.444,0.126289,0.114391,0.243657,NaN,0.116810,0.026726,-0.629032,0.024345,-0.092195,0.007463,0.019048,NaN,0.005102,0.005951,0.243227,0.469075,0.326626
9,1690,1983-03-31,AAPL,0.127908,0.108660,0.000000,0.086197,0.129199,0.063880,NaN,0.457,0.308,41.848,NaN,34.438,0.534,0.020,0.547,9.364,0.133,0.107,NaN,0.399,3.802,9.489,0.045767,0.019868,0.118990,NaN,0.357698,0.158351,-0.130435,0.000000,0.166854,-0.014815,0.000000,NaN,0.012690,0.022318,0.123757,0.326626,0.057759


In [1000]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   gvkey                      15339 non-null  int64         
 1   datadate                   15339 non-null  datetime64[ns]
 2   tic                        15339 non-null  object        
 3   actq                       15338 non-null  float64       
 4   ancq                       15338 non-null  float64       
 5   epsfxq                     15337 non-null  float64       
 6   lctq                       15338 non-null  float64       
 7   ltq                        15338 non-null  float64       
 8   revtq                      15338 non-null  float64       
 9   capxy                      14787 non-null  float64       
 10  roa                        15297 non-null  float64       
 11  roe                        15099 non-null  float64       
 12  pe_i

In [1001]:
# Save DataFrame to CSV file
df = df.sort_values(by=['tic','datadate'])
df.head()
#df.to_csv('processed_data.csv', index=False)  # Set index=False to exclude the index column

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,roa_change,roe_change,pe_inc_change,divyield_change,pcf_change,roce_change,debt_ebitda_change,gpm_change,evm_change,cfm_change,npm_change,PEG_trailing_change,de_ratio_change,curr_ratio_change,ptb_change,relative_quarterly_return,target
0,1690,1980-12-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.283819
1,1690,1981-03-31,AAPL,0.188251,0.072247,0.142857,0.365898,0.404152,0.164801,NaN,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,-0.287897,-0.460829,-0.040696,NaN,0.147918,0.279839,-0.259259,0.016166,0.311636,0.000000,0.040000,NaN,-0.244604,0.238132,0.099702,-0.283819,0.096445
2,1690,1981-06-30,AAPL,0.162343,0.198897,0.312500,0.265953,0.333577,0.151768,NaN,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,-0.110561,-0.170940,-0.549581,NaN,-0.333208,0.345936,0.400000,0.052273,0.058469,0.000000,0.105769,NaN,-0.123810,0.138219,-0.466114,0.096445,-0.298912
3,1690,1981-09-30,AAPL,0.128400,0.315343,-0.095238,0.408022,0.329220,0.076709,NaN,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,-0.133581,-0.030928,-0.233183,NaN,-0.213799,-0.727060,5.500000,0.051836,-0.389708,0.276786,0.026087,NaN,0.187500,-0.122249,-0.130321,-0.298912,0.395990
4,1690,1981-12-31,AAPL,0.097119,0.298983,0.263158,0.194536,0.199391,0.367278,NaN,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,-0.124197,-0.239362,-0.142632,NaN,3.038277,-0.264151,-0.829670,0.032854,0.248343,-0.020979,-0.033898,NaN,-0.084668,0.080780,-0.101255,0.395990,-0.150875


In [1002]:
train_start_date = '1970-01-31'
valid_start_date = '2010-01-31'
test_start_date = '2012-01-31'

train_df = df[(df['datadate'] >= train_start_date) & (df['datadate'] < valid_start_date)]
valid_df = df[(df['datadate'] >= valid_start_date) & (df['datadate'] < test_start_date)]
test_df = df[(df['datadate'] >= test_start_date)]